In [ ]:
library(dplyr)
library(stats)
library(tidyverse)

#plotting
library(gridExtra)
library(grid)
library(ggplot2)
library(lattice)
fig <- function(width, heigth){
 options(repr.plot.width = width, repr.plot.height = heigth)
 }
#Hypothesis testing
library(lmerTest)
library(lme4)
library(car)

#TukeyHSD
library(agricolae)

In [ ]:
df <- read.table(
  "../../data/dfe_alpha_analysis/20210119/bootstrap_values.txt",
  sep="\t", header=FALSE)
column_names <- c("group","alpha", "omega","pFDM", "NeS_0_1", "NeS_1_10", "NeS_10_100", "NeS_100_99")
colnames(df) <- column_names

In [ ]:
df[1:4,]

## Compare alpha, omega-alpha, and pFDM between candidate and non-candidate genes

In [ ]:
plots<-c()
for (i in c("alpha", "omega","pFDM")){
    summary <- df %>% group_by(group) %>% summarize(mean = mean(eval(as.name(i))),
                                                   sd = sd(eval(as.name(i))))
    #print(summary)
    
    p <- ggplot(summary, aes(x=group, y=mean, colour = group, group = 1)) + 
    geom_pointrange(aes(ymin=mean-sd, ymax=mean+sd))+
    theme_classic() +
    scale_color_manual(values=c('red','blue'))+
    labs(x="group", y = i)
    plots[[i]] <- ggplotGrob(p)
    ggsave(paste("../../data/dfe_alpha_analysis/20210119/mean",i,".pdf", sep =""), width = 4, height = 3.5, units = "in")
    }

In [ ]:
fig(8,5)
plt <- grid.arrange(plots[['alpha']], plots[['omega']], plots[['pFDM']], ncol=2, nrow=2)
ggsave(paste("../../data/dfe_alpha_analysis/20210119/mean_values.pdf", sep =""), 
       plot = plt, width = 8, height = 5, units = "in")

## Are alpha, omega-alpha, and pFDM significantly different between groups?
- Perform one way ANOVA
- Check assumptions with qqplot and distribution of variance
### Alpha

In [ ]:
alpha.model <- aov(alpha ~ group, data=df)
summary(alpha.model)
fig(8,3)
par(mfrow=c(1,3))
plot(alpha.model)

tmp <- df %>% mutate(residuals = as.numeric(alpha.model$residuals))
qqnorm(tmp$residuals[tmp$group == 'candidate'], main = 'candidate residuals')
qqnorm(tmp$residuals[tmp$group == 'noncandidate'], main = 'noncandidate residuals')
fig(5,2.5)

p<- ggplot(tmp, aes(x=residuals, color=group, fill = group)) + 
    geom_density(adjust = 0.5, alpha = 0.5) +
    theme_classic()+
    scale_color_manual(values=c('red','blue'))
p

### Omega-alpha

In [ ]:
omega.model <- aov(omega ~ group, data=df)
summary(omega.model)

fig(8,3)
par(mfrow=c(1,3))
plot(omega.model)

tmp <- df %>% mutate(residuals = as.numeric(omega.model$residuals))
qqnorm(tmp$residuals[tmp$group == 'candidate'], main = 'candidate residuals')
qqnorm(tmp$residuals[tmp$group == 'noncandidate'], main = 'noncandidate residuals')

fig(5,2.5)

p<- ggplot(tmp, aes(x=residuals, color=group, fill = group)) + 
geom_density(adjust = 0.5, alpha = 0.5) +
theme_classic()+
scale_color_manual(values=c('red','blue'))
p

### pFDM

In [ ]:
pFDM.model <- aov(pFDM ~ group, data=df)
summary(pFDM.model)
fig(8,3)
par(mfrow=c(1,3))
plot(pFDM.model)

tmp <- df %>% mutate(residuals = as.numeric(pFDM.model$residuals))
qqnorm(tmp$residuals[tmp$group == 'candidate'], main = 'candidate residuals')
qqnorm(tmp$residuals[tmp$group == 'noncandidate'], main = 'noncandidate residuals')

fig(5,2.5)

p<- ggplot(tmp, aes(x=residuals, color=group, fill = group)) + 
geom_density(adjust = 0.5, alpha = 0.5) +
theme_classic() +
scale_color_manual(values=c('red','blue'))
p

#### Distribution of pDFM aov model was non-normal. Use nonparametric equivalent

In [ ]:
wilcox.test(x = df$pFDM[df$group == "candidate"], 
            y=df$pFDM[df$group == "noncandidate"] , 
            alternative = "two.sided", 
            paired = F,
            conf.int = T)

## Are DFEs across NeS bins significanty different between candidate and noncandidate genes?
### Check normal distribution of DFE values

In [ ]:
fig(8,6)
plots = c()
for (i in colnames(df[,seq((ncol(df)-3),ncol(df))])){
    
    p<- ggplot(df, aes(x=eval(as.name(i)), color=group, fill = group)) + 
        geom_density(adjust = 0.5, alpha = 0.5) +
        theme_classic() +
        scale_color_manual(values=c('red','blue'))+
        labs(x="DFE", y = "frequency", title = i)
    plots[[i]] <- ggplotGrob(p)
    
}
            
plt <- grid.arrange(plots[["NeS_0_1"]], plots[["NeS_1_10"]], plots[["NeS_10_100"]], plots[["NeS_100_99"]], ncol=2, nrow=2)

### Plot DFEs 

In [ ]:
tmp <- gather(df, "NeS_0_1", "NeS_1_10", "NeS_10_100", "NeS_100_99", key = "NeS_bins", value = "DFE") %>% 
        select(group, NeS_bins, DFE) %>%
        group_by(group, NeS_bins) %>%
        summarize(mean = mean(DFE),
              sd = sd(DFE))
tmp$NeS_bins <- factor(tmp$NeS_bins, levels = c("NeS_0_1", "NeS_1_10", "NeS_10_100", "NeS_100_99"))
head(tmp)

In [ ]:
fig(6,4)
p<- ggplot(tmp, aes(x=NeS_bins, y=mean, fill=group)) + 
  geom_bar(stat="identity", color="black", 
           position=position_dodge()) +
  geom_errorbar(aes(ymin=mean-sd, ymax=mean+sd), width=.2,
                 position=position_dodge(.9))+
    labs(x="NeS bins", y = "DFE", title = "DFE in NeS bins") +
   theme_classic() +
   scale_fill_manual(values = c('red','blue'))
print(p)

In [ ]:
runs<- row.names(df)
#print(runs)
tmp <- df %>% mutate(run = as.factor(runs)) %>%
    gather(., "NeS_0_1", "NeS_1_10", "NeS_10_100", "NeS_100_99", key = "NeS_bins", value = "DFE")
tmp$NeS_bins <- factor(tmp$NeS_bins, levels = c("NeS_0_1", "NeS_1_10", "NeS_10_100", "NeS_100_99"))
head(tmp)

In [ ]:
runs<- row.names(df)
#print(runs)
tmp <- df %>% mutate(run = as.factor(runs), 
                    group = case_when(group == "candidate" ~ "c",
                                     group == "noncandidate" ~ "nc")) %>%
    gather(., "NeS_0_1", "NeS_1_10", "NeS_10_100", "NeS_100_99", key = "NeS_bins", value = "DFE") %>%
    unite(., group, NeS_bins, col = "factor", sep = "_")

#tmp$NeS_bins <- factor(tmp$NeS_bins, levels = c("NeS_0_1", "NeS_1_10", "NeS_10_100", "NeS_100_99"))
head(tmp)


In [ ]:
print(leveneTest(DFE ~ factor, data = tmp))
variance <- tmp %>% group_by(factor) %>% summarize(variance = var(DFE), mean = mean(DFE), )
variance

In [ ]:
pairwise.t.test(tmp$DFE, tmp$factor,
                 p.adjust.method = "BH", pool.sd = FALSE)
oneway.test(DFE ~ factor, data = tmp)

In [ ]:
#With random effects
aov.model <- aov(DFE ~ group * NeS_bins + Error(run), data=tmp)
summary(aov.model)
tukey <- HSD.test(model, trt = c("group", "NeS_bins"))
tukey
tukey_table <- merge(rownames_to_column(tukey$groups), tukey$means) %>% separate(., rowname, sep = ":", into = c("group", "NeS_bins"))
tukey_table$NeS_bins <- factor(tukey_table$NeS_bins, levels = c("NeS_0_1", "NeS_1_10", "NeS_10_100", "NeS_100_99"))

In [ ]:
fig(6,4)
p<- ggplot(tukey_table, aes(x=NeS_bins, y=DFE, fill=group)) + 
    geom_bar(stat="identity", color="black", 
           position=position_dodge()) +
    geom_errorbar(aes(ymin=DFE-std, ymax=DFE+std), width=.2,
                 position=position_dodge(.9))+
    labs(x="NeS bins", y = "DFE", title = "DFE in NeS bins") +
    geom_text(aes(x = NeS_bins, y = DFE, label = groups),vjust=-2,position=position_dodge(.9)) +
   theme_classic() +
   scale_fill_manual(values = c('red','blue')) 
    
print(p)
ggsave(paste("../../data/dfe_alpha_analysis/20210119/DFE.pdf", sep =""), 
       plot = p, width = 6, height = 4, units = "in")